In [48]:
import os
import pandas as pd

from surprise import Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate

In [41]:
file_path_metadata = "../data/news-portal-user-interactions-by-globocom/articles_metadata.csv"
df_metadata = pd.read_csv(file_path_metadata)
df_metadata = pd.DataFrame(df_metadata)
df_metadata

,article_id,category_id,created_at_ts,publisher_id,words_count
0,0,0,1513144419000,0,168
1,1,1,1405341936000,0,189
2,2,1,1408667706000,0,250
3,3,1,1408468313000,0,230
4,4,1,1407071171000,0,162
...,...,...,...,...,...
364042,364042,460,1434034118000,0,144
364043,364043,460,1434148472000,0,463
364044,364044,460,1457974279000,0,177
364045,364045,460,1515964737000,0,126


In [42]:
file_path_clicks = "../data/news-portal-user-interactions-by-globocom/clicks_sample.csv"
df_click = pd.read_csv(file_path_clicks)
df = pd.DataFrame(df_click)
df_click

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
2,1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2
3,1,1506825426267738,1506825426000,2,96663,1506827047951,4,1,17,1,16,2
4,2,1506825435299739,1506825435000,2,119592,1506827090575,4,1,17,1,24,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1878,705,1506828968165442,1506828968000,2,119592,1506830912301,4,1,17,1,21,2
1879,705,1506828968165442,1506828968000,2,284847,1506830942301,4,1,17,1,21,2
1880,706,1506828979881443,1506828979000,3,108854,1506829027334,4,3,2,1,25,1
1881,706,1506828979881443,1506828979000,3,96663,1506829095732,4,3,2,1,25,1


In [43]:
df_merged = df_click.merge(df_metadata, left_on='click_article_id', right_on='article_id')
df_merged = df_merged[['user_id', 'article_id', 'category_id']]
df_merged

,user_id,article_id,category_id
0,0,157541,281
1,0,68866,136
2,1,235840,375
3,1,96663,209
4,2,119592,247
...,...,...,...
1878,705,119592,247
1879,705,284847,412
1880,706,108854,230
1881,706,96663,209


In [44]:
df_merged["value"] = 1
df_merged

,user_id,article_id,category_id,value
0,0,157541,281,1
1,0,68866,136,1
2,1,235840,375,1
3,1,96663,209,1
4,2,119592,247,1
...,...,...,...,...
1878,705,119592,247,1
1879,705,284847,412,1
1880,706,108854,230,1
1881,706,96663,209,1


In [45]:
grouped = df_merged.groupby(['user_id', 'category_id'])
df_aggregated = grouped['value'].sum().reset_index()
df_aggregated = df_aggregated.rename(columns={'value': 'rate'})
df_aggregated

,user_id,category_id,rate
0,0,136,1
1,0,281,1
2,1,209,1
3,1,375,1
4,2,26,1
...,...,...,...
1707,705,247,1
1708,705,412,1
1709,706,136,1
1710,706,209,1


In [46]:
reader = Reader(rating_scale=(1,5))
df_significant = df_aggregated.loc[df_aggregated["rate"] > 1]
data = Dataset.load_from_df(df_significant[['user_id', 'category_id', 'rate']], reader)

In [47]:
print('We have selects', len(df_significant), 'interactions.')

We have selects 136 interactions.


In [49]:
trainset, testset = train_test_split(data, test_size=0.25)
print('Test set lenght :', len(testset))
print('Train set lenght :', len(df_significant) - len(testset))

Test set lenght : 34
Train set lenght : 102


In [50]:
from surprise import SVD, accuracy
algo = SVD()
algo.fit(trainset)

In [51]:
predictions = algo.test(testset)
print('Number of predictions in Test set :', len(predictions))

Number of predictions in Test set : 34


In [52]:
accuracy.rmse(predictions)

RMSE: 0.4533


0.45331863973933384